# Market Breakdown

## Code

### Prep

Imports

In [1]:
# Structure packages
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import datetime as dt

# Data
import yfinance as yf
import investpy

# Graphing packages
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

# Notebook packages
from IPython.display import display, HTML

Definitions

In [2]:
# Setting dates
no_of_days = 180
today = dt.datetime.now()
time_delta = today - dt.timedelta(days=no_of_days)

today = str(today.strftime("%Y-%m-%d"))
time_delta = str(time_delta.strftime("%Y-%m-%d"))

# Adjusting Display settings
css = """
.output {
    flex-direction: row;
}
"""

HTML("<style>{}</style>".format(css))

# Setting the plotly theme to 'dark'
pio.templates.default = "plotly_dark"

<br>
Assets

In [3]:
# Yield Curves
countries = [
    "united-states",
    "china",
    "united-kingdom",
    "japan",
    "germany",
    "italy",
]

# Equity Indexes
equity_indexes = {
    "MIB (Italy)": "FTSEMIB.MI",
    "DE40 (Germany)": "^GDAXI",
    "FTSE100 (U.K.)": "^FTSE",
    "S&P500 (U.S.)": "^GSPC",
    "Nasdaq Composite (U.S.)": "^IXIC",
    "Nikkei (Japan)": "^N225",
    "Hang Seng (H.K.)": "^HSI",
    "'FIX' China large-cap ETF": "FXI"
}

# Commodity Futures
list_of_futures = [
    "GC=F",  # Gold futures
    "PL=F",  # Platinum futures
    "BZ=F",  # Brent futures
    "NG=F",  # Natural gas futures
    "ZW=F",  # Wheat futures
    "ZC=F",  # Corn futures
]

# Currencies
FX_pairs = {
    "EUR/USD": "EURUSD=X",
    "EUR/CHF": "EURCHF=X",
    "EUR/JPY": "EURJPY=X",
    "EUR/GBP": "EURGBP=X",
    "GBP/USD": "GBPUSD=X",
    "USD/JPY": "USDJPY=X",
    "USD/CHF": "USDCHF=X",
    "USD/AUD": "USDAUD=X",
    "USD/CAD": "USDCAD=X",
}

### Government Bonds

In [4]:
def get_yield_curves(country):
    """
    This function obtains the yield curve data for a given country, converts the data from str to float, and calculates the
    yield curve 1 month ago and 6 months ago.
    """

    # Download the data
    table = pd.read_html(f"http://www.worldgovernmentbonds.com/country/{country}/")[0]

    # Extract the relevant series
    table = table[
        [
            ("ResidualMaturity", "ResidualMaturity"),
            ("Yield", "Last"),
            ("Yield", "Chg 1M"),
            ("Yield", "Chg 6M"),
        ]
    ]

    # Make all "n.a." strings actual NaN values
    table = table.replace("n.a.", np.nan)

    # Convert the "Last" series to float
    table.loc[:, ("Yield", "Last")] = pd.to_numeric(table.loc[:, ("Yield", "Last")].str.strip("%"))

    # Convert the Chg 1M and Chg 6M series to floats
    for col in [("Yield", "Chg 1M"), ("Yield", "Chg 6M")]:
        table.loc[:, col] = pd.to_numeric(
            table.loc[:, col].str.replace("[-+]", "", regex=True).str.replace(" bp", "", regex=False)
        )

    # Divide the Chg series by 100
    table.loc[:, [("Yield", "Chg 1M"), ("Yield", "Chg 6M")]] /= 100

    # Create old yield curves, to be plotted against the latest one
    table.loc[:, ("1mo ago")] = (
        table.loc[:, ("Yield", "Last")] - table.loc[:, ("Yield", "Chg 1M")]
    )
    table.loc[:, ("6mo ago")] = (
        table.loc[:, ("Yield", "Last")] - table.loc[:, ("Yield", "Chg 6M")]
    )

    # Drop the original Chg 1M and Chg 6M columns
    table.drop(columns=[("Yield", "Chg 1M"), ("Yield", "Chg 6M")], inplace=True)

    # Rename the columns
    table.columns = ["ResidualMaturity", "Last", "1mo ago", "6mo ago"]

    # Return the results
    return table

In [5]:
# Process the names
yield_curves = {
    curve.replace("-", " ").title(): get_yield_curves(curve) for curve in countries
}

# Generate the subplot space
yc_fig = make_subplots(
    rows=2,
    cols=3,
    shared_xaxes=False,
    subplot_titles=[name for name in yield_curves.keys()],
)

# Define the colors to use for each series
colors = ["#ef553b", "#6a76f9", "#00cc96"]

# Add traces for each series in each subplot
for i, (name, df) in enumerate(yield_curves.items()):
    for j, col in enumerate(["Last", "1mo ago", "6mo ago"]):
        mode = "lines+markers" if col == "Last" else "lines"
        line_style = "dash" if col != "Last" else None
        yc_fig.add_trace(
            go.Scatter(
                x=df["ResidualMaturity"],
                y=df[col],
                name=col,
                mode=mode,
                line=dict(color=colors[j], dash=line_style),
            ),
            row=i // 3 + 1,
            col=i % 3 + 1,
        )


# Update the layout
yc_fig.update_layout(
    title="<b>Yield Curves over the last 6 months</b>",
    yaxis=dict(title=""),
    height=800,
    legend=dict(orientation="h", y=1.125, x=0.75),
    xaxis=dict(tickangle=90),  # This makes the x-axis labels vertical
)

# Format legend
labels_to_show_in_legend = ["Last", "1mo ago", "6mo ago"]
for i, trace in enumerate(yc_fig["data"]):
    if i >= 3:
        trace["showlegend"] = False
    elif not trace["name"] in labels_to_show_in_legend:
        trace["showlegend"] = False

In [6]:
# Calculate the Italy-German spreads
it_de_spreads = yield_curves["Italy"].copy()
it_de_spreads.iloc[:, 1:] -= yield_curves["Germany"].iloc[:, 1:]

# Create a list of traces for each series
traces = []
for i, col in enumerate(["Last", "1mo ago", "6mo ago"]):
    if col == "Last":
        trace = go.Scatter(
            x=it_de_spreads["ResidualMaturity"],
            y=it_de_spreads[col],
            name=col,
            line=dict(color=colors[i], width=2),
            mode="lines+markers",
        )
    else:
        trace = go.Scatter(
            x=it_de_spreads["ResidualMaturity"],
            y=it_de_spreads[col],
            name=col,
            line=dict(color=colors[i], dash="dash"),
            mode="lines",
        )
    traces.append(trace)

# Create the figure
spread_fig = go.Figure(data=traces)

# Update the layout
spread_fig.update_layout(
    title="<b>Spread between Italian and German Yield Curves over the last 6 months</b>",
    yaxis_title=r"$y_t^{\text{IT}} - y_t^{\text{DE}}$",
    height=500,
)

# Save the plot to an HTML file
spread_fig.write_html("spread_fig.html", auto_open=False)

<br>

### Main Equity Indexes

In [7]:
# Download the equity prices
index_prices = yf.download(list(equity_indexes.values()), start=time_delta)["Adj Close"]

# Forward fill any missing values
index_prices.ffill(inplace=True)

[*********************100%***********************]  8 of 8 completed


In [8]:
# Define the subplot titles
subplot_titles = list(equity_indexes.keys())

# Create the subplot figure
eq_indexes_fig = make_subplots(rows=2, cols=4, subplot_titles=subplot_titles)

# Add traces for each index
for i, index in enumerate(list(equity_indexes.values())):
    row = i // 4 + 1
    col = i % 4 + 1
    eq_indexes_fig.add_trace(
        go.Scatter(x=index_prices.index, y=index_prices[index]), row=row, col=col
    )

# Update the layout
eq_indexes_fig.update_layout(
    height=600,
    title_text=f"<b>Equity Indexes over the last {no_of_days//30} months</b>",
    showlegend=False,
)

# Save the plot to an HTML file
eq_indexes_fig.write_html("equity_indexes.html", auto_open=False)

In [9]:
# Calculate the cumulative returns for each index
index_cum_rets = np.log1p(index_prices.pct_change()).cumsum()
index_cum_rets.columns = list(equity_indexes.keys())

# Plot the results
cum_ret_fig = px.line(index_cum_rets, render_mode="svg").update_layout(
    height=600,
    title_text=f"<b>Cumulative Returns of Equity Indexes over the last {no_of_days // 30} months</b>",
    showlegend=True,
    legend=dict(title_text=""),
    yaxis_title="Cumulative log returns",
    xaxis_title="",
)

<br>

### Commodity Futures

In [10]:
# Download the commodity futures prices
futures_prices = yf.download(list_of_futures, start=time_delta)["Adj Close"]

# Forward-fill any missing values
futures_prices.ffill(inplace=True)

[*********************100%***********************]  6 of 6 completed


In [11]:
# Create subplots
futures_dash = make_subplots(
    rows=3,
    cols=2,
    subplot_titles=("Gold", "Platinum", "Brent", "Natural Gas", "Wheat", "Corn"),
)

# Add traces for commodity
for i, com in enumerate(list_of_futures):
    row = i // 2 + 1
    col = i % 2 + 1
    futures_dash.add_trace(
        go.Scatter(x=futures_prices.index, y=futures_prices[com]), row=row, col=col
    )

# Update layout
futures_dash.update_layout(
    height=800,
    title_text=f"<b>Commodity Futures over the last {no_of_days//30} months</b>",
    showlegend=False,
)

# Save the plot to an HTML file
futures_dash.write_html("futures_dash.html", auto_open=False)

<br>

### Currencies

In [12]:
# Download the historical data
FX_prices = yf.download(list(FX_pairs.values()), start=time_delta)["Adj Close"]

# Forward-fill any missing values
FX_prices.ffill(inplace=True)

[*********************100%***********************]  9 of 9 completed


In [13]:
# Define the subplot titles
subplot_titles = list(FX_pairs.keys())

# Create the subplot figure
FX_fig = make_subplots(rows=3, cols=3, subplot_titles=subplot_titles)

# Add traces for each FX pair
for i, pair in enumerate(list(FX_pairs.values())):
    row = i // 3 + 1
    col = i % 3 + 1
    FX_fig.add_trace(go.Scatter(x=FX_prices.index, y=FX_prices[pair]), row=row, col=col)

# Update the layout
FX_fig.update_layout(
    height=800,
    title_text=f"<b>FX Pairs over the last {no_of_days//30} months</b>",
    showlegend=False,
)

# Save the plot to an HTML file
FX_fig.write_html("FX_pairs.html", auto_open=False)

# Note: if X/Y falls, it means it takes fewer Y to buy X – the value of X is falling and/or the value of Y is rising.

<br>

## Dashboard

In [14]:
# Show the bond plot
yc_fig.show()

# Show the spread plot
spread_fig.show()

# Show the equity indexes plot
eq_indexes_fig.show()

# Show the equity index cumulative returns plot
cum_ret_fig.show()

# Show the commodity futures plot
futures_dash.show()

# Show the FX plot
FX_fig.show()